In [60]:
import sqlite3
import pandas as pd
import plotly.express as px
import numpy as np
import sqlalchemy as sa
from src.models import *

from src.pkmn_constants import *

### Get Data

In [61]:
with Session(engine) as session:

    # current playthrough
    playthrough = (
        session
        .query(Playthrough)
        .order_by(Playthrough.adventure_started.desc())
        .first()
    )
    print(f"Current playthrough: {playthrough}")

    # get event data
    query = (
        session
        .query(Playthrough,Event,TeamMemberEntry)
        .filter(Playthrough.adventure_started == (
            session
            .query(sa.func.max(Playthrough.adventure_started))
            .scalar_subquery()
        ))
        .join(Playthrough.events, isouter=True)
        .join(Event.team_member_entries, isouter=True)
    )
    data = pd.read_sql(query.statement, engine)

    # get pokemon color
    team_member_entries = (
        session
        .query(TeamMemberEntry,TeamMember)
        .filter(TeamMember.playthrough == playthrough)
        .join(TeamMemberEntry.team_member)
        .order_by(TeamMemberEntry.no)
    ).all()
    team_member_color_discrete_map = {}
    team_member_slots = []
    for item in team_member_entries:
        team_member_entry, team_member = item.tuple()

        # get team_member_name
        data.loc[data['team_member_slot'] == team_member.slot, 'team_member'] = team_member.to_str(session)

        # color
        if team_member_entry.type1:
            team_member_color_discrete_map[team_member.to_str(session)] = TYPES[team_member_entry.type1]
data

Current playthrough: Black 2023-06-24


,id_no,name,version,adventure_started,no,playthrough_id_no,location_name,location_region,event_type,event_name,...,no_1,team_member_playthrough_id_no,team_member_slot,event_no,level,species,dex_no,type1,type2,team_member
0,26852,Ben,Black,2023-06-24,4,26852,Accumula Town,Unova,Battle,PKMN Trainer N,...,2.0,26852,1.0,4.0,7.0,None,NaN,None,None,Herdier
1,26852,Ben,Black,2023-06-24,59,26852,Castelia City,Unova,Battle,Clerk F Alberta,...,NaN,None,NaN,NaN,NaN,None,NaN,None,None,NaN
2,26852,Ben,Black,2023-06-24,57,26852,Castelia City,Unova,Battle,Clerk F Ingrid,...,35.0,26852,2.0,57.0,19.0,None,NaN,None,None,Pansear
3,26852,Ben,Black,2023-06-24,58,26852,Castelia City,Unova,Battle,Clerk M Clemens,...,36.0,26852,3.0,58.0,19.0,None,NaN,None,None,Palpitoad
4,26852,Ben,Black,2023-06-24,62,26852,Castelia City,Unova,Battle,Clerk M Ivan,...,NaN,None,NaN,NaN,NaN,None,NaN,None,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158,26852,Ben,Black,2023-06-24,12,26852,Striaton City,Unova,Battle,PKMN Trainer Cheren,...,NaN,None,NaN,NaN,NaN,None,NaN,None,None,NaN
159,26852,Ben,Black,2023-06-24,13,26852,Striaton City,Unova,Battle,Waiter Maxwell,...,9.0,26852,2.0,13.0,11.0,None,NaN,None,None,Pansear
160,26852,Ben,Black,2023-06-24,14,26852,Striaton City,Unova,Battle,Waitress Tia,...,NaN,None,NaN,NaN,NaN,None,NaN,None,None,NaN
161,26852,Ben,Black,2023-06-24,25,26852,Wellspring Cave,Unova,Battle,Team Plasma Grunt,...,NaN,None,NaN,NaN,NaN,None,NaN,None,None,NaN


#### Level Data

In [68]:
df = (
    data
    .pivot_table(values='level',index='no',columns='team_member')
    .reindex(np.arange(data['no'].min(),data['no'].max()+1))
    .ffill()
)
display(df)
df.index
df.columns
(px
    .line(df,
        x=df.index,
        y=df.columns,
        color_discrete_map=team_member_color_discrete_map,
        title=f"Team Member Levels")
    .update_layout(hovermode='x')
    .update_traces(hovertemplate="%{y}")
)

team_member,Archen,Herdier,Palpitoad,Pansear
no,,,,
1,NaN,5.0,NaN,NaN
2,NaN,5.0,NaN,NaN
3,NaN,5.0,NaN,NaN
4,NaN,7.0,NaN,NaN
5,NaN,8.0,NaN,NaN
...,...,...,...,...
147,30.0,31.0,32.0,31.0
148,30.0,31.0,32.0,31.0
149,31.0,31.0,33.0,31.0


## Levels Over Time

In [69]:
# px.line(data, x='')